# Web-Scraping United Nations Webpage:

+ This exercise was used to find what countries voted: `for/against/abstained` for any given `Resolution`

`------------------------------------------------------------------------------`

# `with Mr Fugu Data Science`



# (◕‿◕✿)

[Github](https://github.com/MrFuguDataScience) | [Youtube](https://www.youtube.com/channel/UCbni-TDI-Ub8VlGaP8HLTNw?view_as=subscriber)

# Need to Install:

[BeautifulSoup](https://pypi.org/project/beautifulsoup4/) | [pyPDF2 official docs](https://pypi.org/project/PyPDF2/) | [textract_official docs](https://anaconda.org/conda-forge/textract) | [selenium](https://selenium-python.readthedocs.io/installation.html#downloading-python-bindings-for-selenium)


`conda install -c conda-forge textract` | `conda install -c conda-forge pyPDF2`

or 

`pip install pyPDF2` | `pip install textract`

In [2]:
import pandas as pd
# import numpy as np

import requests  # grab web-page
import pickle
from bs4 import BeautifulSoup as bsopa  #parse web-page 

import re     # regular expressions
import PyPDF2 # pdf reader/parser


# !pip install --upgrade pip

In [3]:
response=requests.get('https://www.un.org/en/ga/74/resolutions.shtml')

response.status_code # checking that the request was (ok=200)

soupie=bsopa(response.text,'lxml')  # converting format

html_source=soupie.find('div',{"class":"col-xs-12 col-md-3"}) # html source code

# tags=html_source.find('a').find_all('href')

tags=html_source.find_all('a') 



In [4]:
def tag_parse(tags):
    # Order (Key_word,href)
    
    g=[]
    for i in tags:
            g.append([''.join(i),i])
            
    # Parse and take only (key_word=='Resolutions',href)
    f=[]
    for j,k in g:
        rt=re_pattern= r'\b(?:Resolutions.*)'
        if re.findall(rt,j):
            f.append([j,k])
            
    # fixing web addresses:
    k=[]
    for i in f:
        k.append([i[0],'https://www.un.org'+i[1]['href']])

    # Only take the web address with general assembly (using greedy method)
    b=[]
    for i in k:    
        re_pattern=r'\b(?:general-assembly.*)'  
        if re.findall(re_pattern,i[1]):
            b.append([i[0],i[1]])
    return b



#Final Result, This webpage will link to where we get ALL Resolutions by Year! (If exists)
tag_parse(tags)
# https://www.un.org/en/sections/documents/general-assembly-resolutions/index.html

[['Resolutions ',
  'https://www.un.org/en/sections/documents/general-assembly-resolutions/index.html']]

 # Now we are going from the First page to the link: 
 + The next page will be buttons for each` Session Year`
 
 `---------------------------`
 + **There is one issue with parsing this** : `soupie_.find_all('div',{"class":"col-sm-2"})`
 *because it is involved in a `nested list` and calls it more than once*. If you don't pay attention you would miss all `YEARS` of resolutions.
 
 + For example: if you do:
 
 `html_source_=soupie_.find('div',{"class":"col-sm-2"})`   (*This line is your issue*)

  `tags_=html_source_.find('ul').find_all('a')`

In [5]:
response_=requests.get(tag_parse(tags)[0][1])

response_.status_code # checking that the request was (ok=200)

soupie_=bsopa(response_.text,'lxml')  # converting format

html_source_=soupie_.find_all('div',{"class":"col-sm-2"}) # html source code

# tags_=html_source_.find('ul').find_all('a')
# html_source_

In [6]:

def parse_UN_reg_sess_pg(html_source_):
    """
    webpages: parse and only take strings including http:
    years= this is the session years
    
    Only using webpage which has the word 'research': '\b(?:research.*)\b'
    
    Match Until First Instance of " : +?(?=\")
    Find Everything with http: (?:http.*), non-greedy ?:, greedy: .*

    """
    webpages=re.findall(r'http?://\S+', str(html_source_))
    years_=re.findall(r'[0-9]{4}',str(html_source_))
    session_number=re.findall(r'[0-9]{2}',str(html_source_))

    o=[]
    for i in webpages:
        re_pattern=r'\b(?:http.*).+?(?=\")\b'

        re_pat_2=r'\b(?:research.*)\b'
        if re.findall(re_pat_2,str(i)):
            
            o.extend(re.findall(re_pattern,i)) # webpage list for webscraping
    
    # 2 parameter output: webpage, list(year,session_num,webpages)
    return o,list(zip(years_,session_number,o)) 


#-----------------
list_gen_assemb_pgs=parse_UN_reg_sess_pg(html_source_)[0]

yr_sess_num_webpage_list =parse_UN_reg_sess_pg(html_source_)[1]

# Showing first three entries: 
list_gen_assemb_pgs[:3]



['http://research.un.org/en/docs/ga/quick/regular/73',
 'http://research.un.org/en/docs/ga/quick/regular/72',
 'http://research.un.org/en/docs/ga/quick/regular/71']

# Now to 3<sup>rd</sup> page:

+ This is a page where we will have up to 3 optional pages to sift through:
    * each page will be a (`Volume:1-3`)

In [7]:
# Do a generator:

def generator_vol_finder(list_gen_assemb_pgs):
    p=[]
    # 'http://research.un.org/en/docs/ga/quick/regular/73'
    for i in list_gen_assemb_pgs:

        response_th=requests.get(i) # loop through list of pages!

        response_th.status_code # checking that the request was (ok=200)

        soupie_th=bsopa(response_th.text,'lxml')  # converting format

        html_source_th=soupie_th.find('div',{"class":"col-md-9"}) # html source code

        tagies_=html_source_th.find_all('div',{"class":"clearfix"})

        p.append(tagies_)
    
    yield p  # Generator, goes 1-by-1 

nm=generator_vol_finder(list_gen_assemb_pgs)
# tagies_
gh=[]
for i in nm:
    gh.append(i)
gh

[[[<div class="clearfix" id="s-lg-content-44351554">
   <p>Resolutions and Decisions adopted by the General Assembly during its 73rd session, GAOR, 73rd session, Supplement No. 49</p>
   <ul>
   <li>Volume I: Resolutions, 18 September - 22 December 2018: <a href="https://undocs.org/a/73/49(Vol.I)">A/73/49 (Vol.I)</a></li>
   <li>Volume II: Decisions, 18 September - 22 December 2018: <a href="https://undocs.org/a/73/49(Vol.II)">A/73/49 (Vol.II)</a></li>
   </ul>
   </div>,
   <div class="clearfix" id="s-lg-content-49665428">
   <p><iframe frameborder="0" height="35000px" scrolling="auto" seamless="seamless" src="//www.un.org/depts/dhl/resguide/r73_resolutions_table_en.htm" style="line-height: 1.6;" width="100%"></iframe></p>
   </div>],
  [<div class="clearfix" id="s-lg-content-41368957">
   <p>Resolutions and Decisions adopted by the General Assembly during its 72nd session, GAOR, 72nd session, Supplement No. 49</p>
   <ul>
   <li>Volume I: Resolutions, 12 September - 24 December 2017:

# We are finding all the `Resolution Pages`:
+ each document will have a *Volume* associated to it (*not all of these files*) have our information.


In [8]:
"""All Webpages with Resolutions: there are appended/corrected files which will not
have any details and I just need to either do try/except or if/else
"""

yy=re.findall(r'https?://\S+', str(gh))
o=[]
for i in yy:
    re_pattern=r'\b(?:http.*).+?(?=")'
#     re_pat=r'\b(?:http.*).+?(?=")'
    if re.findall(re_pattern,str(i)):
        o.extend(re.findall(re_pattern,str(i))) # webpage list for webscraping
o



['https://undocs.org/a/73/49(Vol.I)',
 'https://undocs.org/a/73/49(Vol.II)',
 'http://www.un.org/ga/search/view_doc.asp?symbol=a/72/49(Vol.I)',
 'http://www.un.org/ga/search/view_doc.asp?symbol=a/72/49(Vol.II)',
 'http://www.un.org/ga/search/view_doc.asp?symbol=a/72/49(Vol.III)',
 'http://www.un.org/ga/search/view_doc.asp?symbol=a/71/49(Vol.I)',
 'http://www.un.org/ga/search/view_doc.asp?symbol=a/71/49(Vol.II)',
 'http://www.un.org/ga/search/view_doc.asp?symbol=a/71/49(Vol.III)',
 'http://www.un.org/ga/search/view_doc.asp?symbol=a/70/49(Vol.I)',
 'http://www.un.org/ga/search/view_doc.asp?symbol=a/70/49(Vol.II)',
 'http://www.un.org/ga/search/view_doc.asp?symbol=A/70/49%20(VOL%20III)',
 'http://www.un.org/ga/search/view_doc.asp?symbol=a/69/49(Vol.I)',
 'http://www.un.org/ga/search/view_doc.asp?symbol=A/69/49(Vol.II)',
 'http://www.un.org/ga/search/view_doc.asp?symbol=A/69/49(vol.iii)',
 'http://www.un.org/ga/search/view_doc.asp?symbol=a/68/49(Vol.I)',
 'http://www.un.org/ga/search/view_

# This is the Pdf files stored as `Iframes/frame`:

+ This cannot be directly called in `Beautiful Soup` since it is a *dynamic* webpage with embeded content. 
    + There are a few work arounds:
    
        **1<sup>st</sup> )** : download all files then parse.
        
        **2<sup>nd</sup> )** : find a way to read these without downloading.

Either way there are *`Pros & Cons`* because of the hidden *`Shadow DOM`* and automating this process.

In [9]:
# this is what one of the files looks like, it is an Iframe/frame: 
# (you can scroll this btw if interested)

url='https://undocs.org/a/73/49(Vol.I)'
from IPython.display import IFrame 

if True:
    from IPython.display import IFrame    
    l=display(IFrame(url, width=830, height=560))

#     kk=html.fromstring(str(l))

# Notice the `shadow-root` , this is our problem for webscraping

`______________________________`

![shado_root](shadow_root.png)

# If you try to use `Beautiful Soup` with Iframes it won't get your data. These are dynamic pages now and test software such as `Selenium` can be used.

+ Unfortunately, since the data are hidden by `Shadow DOM: the shadow-root` you see above you will need to get creative. 

In [10]:
# url = 'http://www.un.org/ga/search/view_doc.asp?symbol=a/68/49(Vol.II)'
url='https://undocs.org/pdf?symbol=en/a/73/49(Vol.I)'
# url='https://www.un.org/ga/search/view_doc.asp?symbol=A/64/49(Vol.I)(Supp)'
# url='https://www.un.org/ga/search/view_doc.asp?symbol=a/63/49(Vol.I)(supp)'
response = requests.get(url)
# my_raw_data = bsopa(response.text,'lxml')
# my_raw_data.find_all('frame')

In [11]:
import os,io
url='https://undocs.org/pdf?symbol=en/a/73/49(Vol.I)'
response = requests.get(url)

#Discerning the number of pages will allow us to parse through all the pages.
with io.BytesIO(response.content) as open_pdf_file:
    read_pdf = PyPDF2.PdfFileReader(open_pdf_file)
    num_pages = read_pdf.getNumPages()
    print(num_pages)

    read_pdf=PyPDF2.PdfFileReader(open_pdf_file)
#     num_pages = read_pdf.numPages
    count = 0
    text = ""
#     #The while loop will read each page.
    while count < num_pages:
        pageObj = read_pdf.getPage(count)
        count +=1
        text += pageObj.extractText()


1188


In [12]:
text

'A/\n7\n3\n/49 (Vol. I)\n \n \nResolutions and \nDecisions\n \nadopted by the General Assembly\n \nduring its s\nevent\ny\n-\nthird\n \nsession\n \n \nVolume I\n \n \nResolutions\n \n \n1\n8\n \nSeptember\n \n\n \n2\n2\n \nDecember\n \n20\n1\n8\n \n \nGeneral\n \nAssembly\n \nOfficial Records\n\nS\nevent\ny\n-\nthird\n \nSession\n \nSupplement No. 49\n \n \n \nUnited Nations\n\nNew York\n, 20\n1\n9\n \n \nNOTE\n \nThe resolutions and decisions of the General Assembly are identified as follows:\n \n \nRegular sessions\n \n \nUntil \nthe thirtieth regular session, the resolutions of the General Assembly were \nidentified by an arabic numeral followed by a roman numeral in parentheses indicating \nthe session (for example: resolution 3363\n \n(XXX)). When several resolutions were \nadopted under \nthe same number, each of them was identified by a capital letter placed \nbetween the two numerals (for example: resolution 3367\n \nA\n \n(XXX), resolutions 3411\n \nA \nand B\n \n(XXX), resolu

In [124]:
D=text

D=D.replace("\n",'').replace('\r','').replace('   ','').replace('..','').strip()


In [125]:
D=D.split('.')


# Create Parsed list with `Resolution & Vote ` strings

+ This list needs to be addressed since it doesn't have resolutions where countries voted.
That is why I had to parse further!

In [15]:
def filter_UN_txt(h):
    b=[]
    for i in h:
        ''' pattern match using (|) "or" operator , greedy form for sentences
        '''
        re_pattern=r'\b(?:RESOLUTION.*)|(?:In favour.*)|(?:Against.*)|(?:Abstaining.*)+\b'
        b.append(re.findall(re_pattern,i))
    return list(filter(None,b))

X=filter_UN_txt(D)

VV=[]
for i in X:
    for j in i:
        VV.extend([j.replace('?'," ")])
print(VV)



['In favour:  Afghanistan, Algeria, Angola, Argentina, Armenia, Azerbaijan, Bahrain, Bangladesh, Barbados, Belarus, Belize, Bhutan, Bolivia (Plurinational State of), Bosnia and Herzegovina, Brunei Darussalam, Burkina Faso, Burundi, Cabo Verde, Cambodia, Central African El Salvador, Equatorial Guinea, Eritrea, Ethiopia, Gambia, Ghana, Grenada, Guatemala, Guinea, Guinea-Bissau, Guyana, Honduras, India, Indonesia, Iran (Islamic Republic of)Democratic Republic, Lebanon, Lesotho, Liberia, Libya, Madagascar, Malawi, Malaysia, Maldives, Mali, Mauritania, Mauritius, Mexico, Mongolia, Morocco, Mozambique, Myanmar, Namibia, Nepal, Nicaragua, Niger, Nigeria, Oman, Pakistan, Palau, Panama, Paraguay, Peru, Philippines, Qatar, Russian Federation, Saint Lucia, Saint Vincent and the Grenadines, Samoa, Sao Tome and Principe, Saudi Arabia, Serbia, Seychelles, Sierra Leone, Singapore, South Africa, South Sudan, Sri Lanka, Sudan, Suriname, Syrian Arab Republic, Tajikistan, Thailand, Timor-Leste, Togo, Tri

In [17]:
# r='RESOLUTION 73/211'
o=[]
for i in VV:
    if re.findall('(?:RESOLUTION.*)',i):
        o.append(re.findall('(?:RESOLUTION.*.+?(?=\ A))',i))
    if re.findall('(?:In favour.*)',i):
        o.append(re.findall('(?:In favour.*)',i))
o

[['In favour:  Afghanistan, Algeria, Angola, Argentina, Armenia, Azerbaijan, Bahrain, Bangladesh, Barbados, Belarus, Belize, Bhutan, Bolivia (Plurinational State of), Bosnia and Herzegovina, Brunei Darussalam, Burkina Faso, Burundi, Cabo Verde, Cambodia, Central African El Salvador, Equatorial Guinea, Eritrea, Ethiopia, Gambia, Ghana, Grenada, Guatemala, Guinea, Guinea-Bissau, Guyana, Honduras, India, Indonesia, Iran (Islamic Republic of)Democratic Republic, Lebanon, Lesotho, Liberia, Libya, Madagascar, Malawi, Malaysia, Maldives, Mali, Mauritania, Mauritius, Mexico, Mongolia, Morocco, Mozambique, Myanmar, Namibia, Nepal, Nicaragua, Niger, Nigeria, Oman, Pakistan, Palau, Panama, Paraguay, Peru, Philippines, Qatar, Russian Federation, Saint Lucia, Saint Vincent and the Grenadines, Samoa, Sao Tome and Principe, Saudi Arabia, Serbia, Seychelles, Sierra Leone, Singapore, South Africa, South Sudan, Sri Lanka, Sudan, Suriname, Syrian Arab Republic, Tajikistan, Thailand, Timor-Leste, Togo, Tr

In [18]:
    
def parse_resol_votes(regex_fcn,lst_):
    gg=[]        
    for j in range(len(lst_)):
#         re_pattern=r'\b(?:In favour.*)\b'
        if (re.findall(regex_fcn,str(lst_[j]))):
            gg.extend(re.findall(regex_fcn,str(lst_[j])))
        else:
            gg.append([])
    return gg

re_pattern=r'\b(?:In favour.*)\b'
re_patt=r'\b(?:RESOLUTION.*)+\b'

votes_=parse_resol_votes(re_pattern,o)

resolu=parse_resol_votes(re_patt,o)


Need to take the current element and compare: 

`[]
 'RESOLUTION 73/91',
 [],
 'RESOLUTION 73/93',
 []`
 
 if a resolution occurs before an empty list append, this is because the next item will be votes sequentially.

In [130]:
def find_resol_we_need(resolu):
    l=[]
    for i in range(len(resolu)-1):
        if resolu[i]==str(resolu[i]) and resolu[i+1]==[]:
            l.append(resolu[i])
    return l

find_resol_we_need(resolu)

['RESOLUTION 73/91',
 'RESOLUTION 73/93',
 'RESOLUTION 73/95',
 'RESOLUTION 73/97',
 'RESOLUTION 73/98',
 'RESOLUTION 73/99',
 'RESOLUTION 73/103',
 'RESOLUTION 73/104',
 'RESOLUTION 73/105',
 'RESOLUTION 73/122',
 'RESOLUTION 73/123',
 'RESOLUTION 73/141',
 'RESOLUTION 73/151',
 'RESOLUTION 73/157',
 'RESOLUTION 73/158',
 'RESOLUTION 73/159',
 'RESOLUTION 73/165',
 'RESOLUTION 73/167',
 'RESOLUTION 73/169',
 'RESOLUTION 73/170',
 'RESOLUTION 73/171',
 'RESOLUTION 73/173',
 'RESOLUTION 73/175',
 'RESOLUTION 73/181',
 'RESOLUTION 73/182',
 'RESOLUTION 73/187',
 'RESOLUTION 73/262',
 'RESOLUTION 73/263',
 'RESOLUTION 73/264']

# I cannot use the above function for `Votes`:
because it will print the first items incorrectly, we need to do a different comparison and look at the current, previous and next element. `This is because what we need are the previous and current not to be strings and the current to be empty and next to be a string.`


In [22]:
from itertools import tee, islice, chain

def previous_and_next(some_iterable):
    prevs, items, nexts = tee(some_iterable, 3)
    prevs = chain([None], prevs)
    nexts = chain(islice(nexts, 1, None), [None])
    return zip(prevs, items, nexts)

In [136]:
# list(previous_and_next(gg))
u=[]
for previous, item, nxt in previous_and_next(votes_):
    if previous==[] and item ==str(item):
        u.append(item)


In [31]:
# Creating a list with tuples [(resolution,votes)...]

resolution_lst=list(zip(find_resol_we_need(resolu),u))

In [32]:
resolution_lst

[('RESOLUTION 73/91',
  'In favour:  Afghanistan, Albania, Algeria, Andorra, Angola, Antigua and Barbuda, Argentina, Armenia, Australia, Austria, Azerbaijan, Bahamas, Bahrain, Bangladesh, Barbados, Belarus, Belgium, Belize, Benin, Bhutan, Bolivia (Plurinational State of), Bosnia and Herzegovina, Botswana, Brazil, Brunei Darussalam, Bulgaria, Burkina Faso, Burundi, Cabo Verde, Cambodia, Central African Republic, Chile, China, Colombia, Comoros, Congo, Costa Rica, Croatia, Cuba, Cyprus, Czechia, Democratic El Salvador, Estonia, Ethiopia, Finland, France, Gabon, Gambia, Georgia, Germany, Ghana, Greece, Grenada, Guinea, Guinea-Bissau, Guyana, Haiti, Honduras, Hungary, Iceland, India, Indonesia, Iran (Islamic Republic of), Iraq, Ireland, Italy, Jamaica, Japan, Jordan, Kazakhstan, Kenya, atic Republic, Latvia, Lebanon, Lesotho, Libya, Liechtenstein, Lithuania, Luxembourg, Malaysia, Maldives, Mali, Malta, Mauritania, Mauritius, Mexico, Monaco, Mongolia, Montenegro, Morocco, Mozambique, Namibi

# Getting `Iframe` that stores the table I need for what Resolution Numbers mean!

In [84]:
url='http://research.un.org/en/docs/ga/quick/regular/72'
# response = requests.get(url)
# response_.status_code # checking that the request was (ok=200)

from selenium import webdriver
browser = webdriver.Chrome('/Users/zatoichi59/Desktop/chromedriver')
browser.get(url)
soup = bsopa(browser.page_source, "lxml")
browser.close()
for x in soup.find_all('iframe'):
    print(x)


resolution_names_table_link=x.get_attribute_list('src')

<iframe frameborder="0" height="35000px" scrolling="auto" seamless="seamless" src="//www.un.org/depts/dhl/resguide/r72_resolutions_table_en.htm" style="line-height: 1.6;" width="100%"></iframe>


# Getting The Resolution Number and its Name:

In [138]:

for i in x.get_attribute_list('src'):
    print('http:'+i)
    response = requests.get('http:'+i)
    my_raw_data = bsopa(response.text,'lxml')
my_tbl=my_raw_data.find('table')
tbl_rows=my_tbl.find_all('tr')
ap=[]
for tr in tbl_rows:
    td= tr.find_all('td')
    row = [i.text for i in td]
#     print(row)
    ap.append(row)


http://www.un.org/depts/dhl/resguide/r72_resolutions_table_en.htm


In [118]:
pp=[]
for i in ap:
    if i !=[]:
        pp.append([i[0],i[-1]])
print(pp)

[['Document links will work once the document has been published in the Official Document System. ', 'Document links will work once the document has been published in the Official Document System. '], ['A/RES/72/313', 'Revitalization of the work of the General Assembly'], ['A/RES/72/312', 'United Nations action on sexual exploitation and abuse'], ['A/RES/72/311', 'Implementation of the recommendations contained in the report of the Secretary-General on the causes of conflict and the promotion \r\n\tof durable peace and sustainable development in Africa'], ['A/RES/72/310', "New Partnership for Africa's Development: progress in implementation and international support"], ['A/RES/72/309', 'Consolidating gains and accelerating efforts to control and eliminate malaria in developing countries, particularly in Africa, by 2030'], ['A/RES/72/308', 'Modalities for the Intergovernmental Conference to Adopt the Global Compact for Safe, Orderly and Regular Migration'], ['A/RES/72/307', 'Modalities 

# Citations:    

# ಠ_ಠ

https://stackoverflow.com/questions/7124778/how-to-match-anything-up-until-this-sequence-of-characters-in-a-regular-expres

https://medium.com/better-programming/how-to-convert-pdfs-into-searchable-key-words-with-python-85aab86c544f

https://stackoverflow.com/questions/5364357/python-parsing-a-string-for-urls-and-extracting-them

https://www.techbeamers.com/switch-between-iframes-selenium-python/

https://www.codementor.io/blog/python-web-scraping-63l2v9sf2q (thorough web-scrape ideas)

https://www.youtube.com/watch?v=jeN9WeTowGg&t=369s

https://stackoverflow.com/questions/51206639/extract-the-text-from-url-which-is-pdf

https://www.youtube.com/watch?v=sNEcpw8LPpo&t=68s

https://www.brodrigues.co/blog/2018-06-10-scraping_pdfs/

https://www.seleniumeasy.com/selenium-tutorials/accessing-shadow-dom-elements-with-webdriver

https://stackoverflow.com/questions/57158681/download-the-file-which-has-stream-url-is-the-chrome-extension-in-the-embed-tag

https://medium.com/rate-engineering/a-guide-to-working-with-shadow-dom-using-selenium-b124992559f

https://help.apify.com/en/articles/2619262-how-to-scrape-pages-with-shadow-dom

https://gist.github.com/ChadKillingsworth/d4cb3d30b9d7fbc3fd0af93c2a133a53

https://stackoverflow.com/questions/1011938/python-loop-that-also-accesses-previous-and-next-values

https://www.google.com/search?q=web+scrape+table+python&oq=wescrape+tabl&aqs=chrome.1.69i57j0l7.8352j0j7&sourceid=chrome&ie=UTF-8#kpvalbx=_FTyfXpqOIYaPtAbq4KygDw39